In [1]:
from pianoq.misc.mplc_writeup_imports import *
from pianoq.misc.mplc_lab_imports import * 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    
dz = 5 
deadzone = 250e-6
plane_10_tilt = 20 

# WFM

In [21]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path = fr'{dir_path}\{timestamp}_QKD_dead_zone_{deadzone*1e6}um_dz10_{dz}mm.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_QKD_dead_zone_{deadzone*1e6}um_dz10_{dz}mm.mplcsim'
# mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, 
#                                     dead_middle_zone=deadzone, last_plane_extra_dist=dz*1e-3)
mplc_sim = create_WFM_QKD_masks(N_iterations=15, out_path=masks_path,
                                dead_middle_zone=deadzone, last_plane_extra_dist=dz*1e-3)
mplc_sim.res._calc_fidelity()
print(f'{mplc_sim.res.fidelity=}')

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087, 0.087, 0.087, 0.087, 0.138, 0.087, 0.087, 0.087, 0.087,
       0.092]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}


100%|██████████| 15/15 [02:12<00:00,  8.81s/it]


mplc_sim.res.fidelity=0.9783116106895021


In [24]:
mplc_sim.res.show_field_intensity((np.abs(mplc_sim.res.input_modes)**2).sum(axis=0))
mplc_sim.res.show_field_intensity((np.abs(mplc_sim.res.output_modes)**2).sum(axis=0))

In [19]:
mplc_sim.res.show_all(1)

# Hardware

In [4]:
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=plane_10_tilt)
integration_time = 1
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

God timetagger


In [5]:
backlash = 0
wait_after_move = 0.2
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)

backlash = 0
wait_after_move = 0.2
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)

Got Zaber motors
Got Thorlabs motors


# Single counts

In [13]:
# Set MPLC 
# dummy old phases path meanwhile  
phases_path = rf'G:\My Drive\Projects\MPLC\results\lab\2024_10_30_make_mirror_work\2024_10_31_10_15_02_fine_new_mirror_from_zero_more.phases'
masks_path = fr'G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_29_10_54_23_U1U_dead_zone_400pix_dz10_5mm.masks'

masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.plane_10_tilts = plane_10_tilt
mplc.load_masks(masks)
time.sleep(0.5)
mplc.restore_location()

In [64]:
resolution = 1
integration_time = 1 

# Scan! 
scan_saveto_path = rf'{dir_path}\{tnow()}_signal_dead_zone_{deadzone}um_dz10_{dz}mm_pl10tilt_{plane_10_tilt}_res_{resolution}_d_mirror.scan'
start_x = 9
end_x = 13
start_y = 6
end_y = 10

x_pixels = 160 // resolution
y_pixels = 160 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path) 
tt = get_timetagger(integration_time=integration_time)  # important to have a new tt 
t1 = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)

time.sleep(2)

scan_saveto_path = rf'{dir_path}\{tnow()}_idler_dead_zone_{deadzone}um_dz10_{dz}mm_pl10tilt_{plane_10_tilt}_res_{resolution}_d_mirror.scan'

start_x = 9.25
end_x = 9.75
start_y = 3.6
end_y = 5.6

x_pixels = 20 // resolution
y_pixels = 80 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)
tt2 = get_timetagger(integration_time=integration_time)
t2 = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt2)
t1.join()
send_telegram_message(f'Done one singles scan')
# t2.join()    
send_telegram_message(f'Done second singles scan')

{'ok': True,
 'result': {'message_id': 71,
  'from': {'id': 7897393437,
   'is_bot': True,
   'first_name': 'complexphoton',
   'username': 'complexphoton_bot'},
  'chat': {'id': 476169345,
   'first_name': 'Ronen',
   'last_name': 'Shekel',
   'username': 'RonenShekel',
   'type': 'private'},
  'date': 1738159643,
  'text': 'Done second singles scan'}}

## Manual motors

In [31]:
mxi.move_absolute(9.47)
myi.move_absolute(3.89)
display(mxi.get_position())
display(myi.get_position())

9.47

3.89

In [89]:
mxs.move_absolute(11.07)
mys.move_absolute(8.1)
display(mxs.get_position())
display(mys.get_position())

11.070002625

8.100012375